## Concepts

### Entropy 
+ Entropy is a measurement of the impuraty and uncertainty in a group of observations. 
+ The larger the entropy is, the more impure and more uncertain the observations are.
+ Formula: $H(p) = -\sum\limits_{i=0}^n{p_{i}\log{p_{i}}}$

### Information gain
+ The difference between emprical entropy and emprical conditional entropy
+ Formula: $g(D, A) = H(D) - H(D|A)$ where A is a given feature

## Algorithom for information gain calculation

Given that training data set $D$, we use $|D|$ to represents the sample size. Suppose that we have $K$ classes(labels) $C_{k}$, k=1,2,...K, $|C_{k}|$ represents the sample size of subset in class k, so we know that $\sum\limits_{k=1}^K|C_{k}| = |D|$. Assume that feature A have n unique values ${a_1, a_2, ...... a_n}$. Based on the value of feature A we can split $D$ into $n$ subsets: $D_1, D_2, D_3, ...... D_n$, we use $|D_i|$ to represents sample size of $D_i$, therefore, we have $\sum\limits_{i}^n|D_i| = |D|$. Mark the sample set belonging to subset $D_{i}$ in class $C_k$ as $D_{ik}$, so $D_{ik} = D_{i} \cap C_{k}$, $|D_{ik}|$ is the sample size of $D_{ik}$, Information Gain Algorithm is as follow:

Tip: In short, given data set $D$, we first split it based on the values of feature $A$ to get $D_i$, and then split $D_i$ based on class $C_k$ and then compute Infomation Gain

## Decision Tree Models : ID3, C4.5, CART

In [12]:
import gc
import sys

In [5]:
import sklearn
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
import typing 

iris_data, target = load_iris(return_X_y = True, as_frame=True)
iris_data = pd.concat([iris_data, target], axis=1)
iris_data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


### ID3

In [8]:
# a function that helps split data set on column level

def df_split(df : pd.DataFrame, col : str)->dict[str, pd.DataFrame]:
    '''
    input:
        df : data set
        label : name of column for label
    output:
        res_dict : a dict containing all sub set that are splitted
    '''
    res_dict = {elem : pd.DataFrame for elem in df[col].unique()}
    
    for label_ in res_dict.keys():
        res_dict[label_] = df[:][df[col] == label_]
    
    return res_dict

# a test case, test if the function is working well
res_dict = df_split(iris_data, 'target')
print(len(res_dict))
print(iris_data['target'].nunique())
del res_dict
gc.collect()

3
3


0

In [11]:
# function for entropy calculation
from math import log


def entropy(elements : typing.List):
    probs = [elements.count(e)/len(elements) for e in set(elements)]
    entro = -1 * sum([prob * log(prob, 2) for prob in probs])
    return entro

# a test case 
print(entropy([1,1,1,2,2,2]))
print(-0.5 * log(0.5, 2) * 2)

1.0
1.0


In [13]:
# a fucntion that select the feature which can maximize the infomation gain
def choose_best_feature(df : pd.DataFrame, label : str):
    '''
    input:
        df : data set
        label : label or target of the data set
    output:
        max_value : maximum information gain
        best_feature : string, name of the best feature
        max_splited : splitted data set using the best feature
    '''
    # compute entropy on label
    entropy_D = entropy(df[label].tolist())
    
    # create feature set
    cols = [col for col in df.columns if col not in [label]]
    
    #initialize infomation gain, beast feature for splitting, and best splitted samples    
    max_value, best_feature = -sys.maxsize, None 
    max_splitted = None
    
    # traverse features
    for col in cols:
        # split data set by feature
        splitted = df_split(df, col)
        # initialize conditional entropy
        entropy_DA = 0
        for subset_col, subset in splitted.items():
            # compute the entropy of subset for given feature
            entropy_Di = entropy(subset[label].tolist())
            # compute the conditional entory of the feature
            entropy_DA += len(subset) / len(df) * entropy_Di
        # compute infomation gain
        info_gain = entropy_D - entropy_DA
        
        if(info_gain > max_value):
            max_value = info_gain
            best_feature = col
            max_splitted = splitted
    
    return max_value, best_feature, max_splitted


In [14]:
## Build a ID3 Class
class ID3Tree():
    
    class TreeNode():
        # define a tree node structure
        def __init__(self, name) -> None:
            self.name = name
            self.connections = {}
        # define a method 'connect' which add child node to a node
        def connect(self, label, node):
            self.connections[label] = node
            
    def __init__(self, df : pd.DataFrame, label : str) -> None:
        self.columns = df.columns
        self.df = df 
        self.label = label 
        self.root = self.TreeNode('Root')
    
    # define an api for constructing tree
    def construct_tree(self):
        self.construct(self.root, '', self.df, self.columns)
        
    def construct(self, parent_node, parent_label, sub_df, columns):
        max_value, best_feature, best_splitted = choose_best_feature(sub_df[columns], self.label)
        
        if not best_feature:
            node = self.TreeNode(sub_df[self.label].iloc[0])
            parent_label.connect(parent_label, node)
            return 
        
        node = self.TreeNode(best_feature)
        parent_node.connect(parent_label, node)
        
        new_columns = [col for col in columns if col != best_feature]
        
        for splitted_value, splitted_data in best_splitted.items():
            self.construct(node, splitted_value, splitted_data, new_columns)